In [76]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns 

In [77]:
stock_data = pd.read_csv('adobe-stock-data.csv')
analyst_data = pd.read_csv('adobe-analyst-data.csv')
analyst_data = analyst_data.rename(columns={'analytst':'analyst'})
analyst_data['date'] = pd.to_datetime(analyst_data['date'])
stock_data = stock_data.rename(columns={'Date':'date'})
stock_data['date'] = pd.to_datetime(stock_data['date'])
stock_data_needed = stock_data[['date', 'Close', 'High', 'Low']]

In [78]:
analyst_data.at[27, 'price_target_after'] = 650
analyst_data.at[27, 'price_target_before'] = 720 
analyst_data.at[255, 'price_target_after'] = 520
analyst_data.at[255, 'price_target_before'] = 460
analyst_data.at[334, 'price_target_after'] = 570
analyst_data.at[334, 'price_target_before'] = 440
analyst_data.at[537, 'price_target_after'] = 358
analyst_data.at[537, 'price_target_before'] = 350

In [79]:
analyst_data['price_target_before'] = pd.to_numeric(analyst_data['price_target_before'], errors='coerce')
analyst_data['price_target_after'] = pd.to_numeric(analyst_data['price_target_after'], errors='coerce')
analyst_data_cleaned = analyst_data.dropna(subset=['price_target_after'])

In [80]:
analyst_data_date_sort = analyst_data_cleaned.sort_values(by='date', ascending=False)

In [81]:
analyst_data_date_sort.head()

,date,company_Name,ticker,broker,analyst,rating_before,rating_after,price_target_before,price_target_after
728,2025-09-05,Adobe Systems Inc,ADBE,NaN,SAKET KALIA,Overweight,Overweight,460.0,460.0
329,2025-08-29,Adobe Systems Inc,ADBE,NaN,KARL KEIRSTEAD,Neutral,Neutral,400.0,400.0
228,2025-06-20,Adobe Systems Inc,ADBE,NaN,GIL LURIA,Buy,Buy,500.0,500.0
805,2025-06-16,Adobe Systems Inc,ADBE,CITI,TYLER RADKE,NEUTRAL,NEUTRAL,NaN,450.0
244,2025-06-13,Adobe Systems Inc,ADBE,MIZUHO,GREGG MOSKOWITZ,OUTPERFORM,OUTPERFORM,575.0,530.0


In [82]:
adobe_data = pd.merge(left=analyst_data_date_sort, right=stock_data_needed, on='date', how='inner')

In [83]:
adobe_data.head()

,date,company_Name,ticker,broker,analyst,rating_before,rating_after,price_target_before,price_target_after,Close,High,Low
0,2025-09-05,Adobe Systems Inc,ADBE,NaN,SAKET KALIA,Overweight,Overweight,460.0,460.0,348.97,354.67,345.10
1,2025-08-29,Adobe Systems Inc,ADBE,NaN,KARL KEIRSTEAD,Neutral,Neutral,400.0,400.0,356.70,357.25,352.46
2,2025-06-20,Adobe Systems Inc,ADBE,NaN,GIL LURIA,Buy,Buy,500.0,500.0,376.92,382.29,373.76
3,2025-06-16,Adobe Systems Inc,ADBE,CITI,TYLER RADKE,NEUTRAL,NEUTRAL,NaN,450.0,401.73,405.00,391.11
4,2025-06-13,Adobe Systems Inc,ADBE,MIZUHO,GREGG MOSKOWITZ,OUTPERFORM,OUTPERFORM,575.0,530.0,391.68,399.85,383.75


In [86]:
def days_to_hit_target(analyst_df, price_df):
    price_df = price_df.sort_values("date")

    def compute_days(row):
        start_date = row["date"]
        target = row["price_target_after"]

        future_prices = price_df[price_df["date"] > start_date].sort_values("date")

        if future_prices.empty:
            return np.nan

        if target >= row["Close"]:
            hit = future_prices[future_prices["High"] >= target]

        else:
            hit = future_prices[future_prices["Low"] <= target]

        if hit.empty:
            return np.nan

        hit_date = hit.iloc[0]["date"]
        return len(future_prices[future_prices["date"] <= hit_date])

    analyst_df["days_to_target"] = analyst_df.apply(compute_days, axis=1)
    return analyst_df


In [88]:
days_to_hit_target(adobe_data, stock_data).tail()

,date,company_Name,ticker,broker,analyst,rating_before,rating_after,price_target_before,price_target_after,Close,High,Low,days_to_target
779,2013-09-18,Adobe Systems Inc,ADBE,DEUTSCHE BANK,NANDAN AMLADI,,BUY,52.0,58.0,52.58,52.67,51.06,61.0
780,2013-06-19,Adobe Systems Inc,ADBE,DEUTSCHE BANK,NANDAN AMLADI,BUY,BUY,50.0,52.0,45.78,46.44,45.35,63.0
781,2013-03-20,Adobe Systems Inc,ADBE,NaN,MATTHEW HEDBERG,Outperform,Outperform,52.0,52.0,42.46,43.23,42.00,126.0
782,2012-12-14,Adobe Systems Inc,ADBE,NaN,MATTHEW HEDBERG,Outperform,Outperform,42.0,42.0,37.56,38.10,37.00,64.0
783,2012-12-14,Adobe Systems Inc,ADBE,JMP,PATRICK WALRAVENS,OUTPERFORM,UNDERPERFORM,NaN,42.0,37.56,38.10,37.00,64.0
